In [1]:
import sys
import os
import numpy as np
import multiprocessing
import pickle
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
from statsmodels.api import OLS, add_constant
import time
from scipy.stats import norm
import datetime as dt

module_path = os.path.abspath(os.path.join('.'))
sys.path.append(module_path + "/../../notebooks/vax_sims/")
from vax_sims_LHS_samples import *
from plot_utils import *
sys.path.append(module_path + "/../../src/simulations_v2")
from multi_group_simulation import MultiGroupSimulation

In [2]:
UNCERTAINTY_PARAMS = ['vax_susc_mult', 'vax_transmission_mult', 'contacts_per_day_mult', 'outside_infection_rate_mult',
                      'cases_isolated_per_contact_trace', 'initial_ID_prevalence']

UNCERTAINTY_PARAM_RANGES = {
    'vax_susc_mult': (0.097608, 0.941192), # 0.5194 +/- 1.96 * 0.2152
    'vax_transmission_mult': (0.25, 1),
    'contacts_per_day_mult': (0.9,2.7),
    'outside_infection_rate_mult': (1, 5),
    'cases_isolated_per_contact_trace': (0.5,1.5),
    'initial_ID_prevalence': (0.003, 0.0054)
}

def load_posterior_df():
    df = pd.read_csv('../vax_sims/posterior_csvs/21_10_08_14:44_posteriors.csv')
    return df

df = load_posterior_df()
MLE_point = list(df.sort_values('posterior', ascending=False).head(1)[UNCERTAINTY_PARAMS].iloc[0])

In [5]:
group_params = load_params("../vax_sims/vax_sim_nominal_params/ug_greek_athlete_nominal.yaml")[1]

group_params['daily_outside_infection_p'] = group_params['daily_outside_infection_p'] * MLE_point[3] * 10
group_params['cases_isolated_per_contact'] *= MLE_point[4]

high_contact_params = group_params.copy()
low_contact_params = group_params.copy()

# For now GA/UG other components only
contact_matrix = np.matrix([[0.736, 0.023], [.028, .148]]) * 10

vax_susc_mult = MLE_point[0]
vax_trans_mult = MLE_point[1]

omicron_multiplier = 5
social_distance_multiplier = 1
contact_matrix = contact_matrix * vax_susc_mult * vax_trans_mult * MLE_point[2] * omicron_multiplier * social_distance_multiplier

high_contact_params['expected_contacts_per_day'] = contact_matrix[0,0]
low_contact_params['expected_contacts_per_day'] = contact_matrix[1,1]

high_contact_params['initial_ID_prevalence'] = 0.04
low_contact_params['initial_ID_prevalence'] = 0.04

# 66% of (3533+8434) ~= 8018
high_contact_params['population_size'] = 3533
low_contact_params['population_size'] = 8434
# Arbitrary testing policy
high_contact_params['test_population_fraction'] = 3/7
low_contact_params['test_population_fraction'] = 3/7

In [6]:
sim = MultiGroupSimulation([high_contact_params, low_contact_params], contact_matrix)

In [9]:
sim_results = list()
for _ in range(100):
    sim_results.append(sim.run_new_trajectory(112)[1])

In [ ]:
pickle.dump(sim_results, open('UG_virtual_sim_results.p', 'wb'))